# Jobs Analysis

## Setup

### Installs

In [1]:
%load_ext rpy2.ipython

In [2]:
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import sys
sys.path.append("/content/drive/Shareddrives/Working Group - NLP in Engineering Education Research/Fall 2021 Independent Study/Sample code")

In [4]:
!git clone https://github.com/crabcamp/lexrank.git

fatal: destination path 'lexrank' already exists and is not an empty directory.


In [5]:
# needed for lexrank
!pip install urlextract

In [6]:
import sys
sys.path.append('/content/lexrank/')

from lexrank.lexrank import degree_centrality_scores

In [7]:
!pip install -U spacy

In [8]:
!pip install umap-learn
!pip install hdbscan
!pip install -U sentence-transformers

In [9]:
!pip install keybert

  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
thinc 8.0.15 requires typing-extensions<4.0.0.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.1.1 which is incompatible.
spacy 3.2.4 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.1.1 which is incompatible.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.


### Imports

In [10]:
import embed_cluster as ec

sklearn modules imported


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


transformer module imported


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [11]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import umap
#import umap.plot
import hdbscan
import spacy
from spacy.lang.en import English

from sklearn.manifold import MDS, TSNE

from textblob import TextBlob
import nltk
nltk.download('punkt')


import pickle

from keybert import KeyBERT


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# this is the code to add download the en_core_web_lg language model for spacy
!python -m spacy download en_core_web_lg
#https://stackoverflow.com/questions/56927602/unable-to-load-the-spacy-model-en-core-web-lg-on-google-colab

# note: need to restart runtime after downloading

     |████████████████████████████████| 777.4 MB 6.3 kB/s 
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
rich 12.2.0 requires typing-extensions<5.0,>=4.0.0; python_version < "3.9", but you have typing-extensions 3.10.0.2 which is incompatible.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


## Utility Functions

In [13]:
def select_and_filter(original_df, text_column):
    """
    Parameters
    ----------
    item : str
        Name of the column to filter based on.
    original_df : dataframe
        Original input dataframe with text columns.

    Returns
    -------
    filtered_df : dataframe
        Dataframe that has removed the NAs in the {item} column.
    item_list : list
        List of the text in the {item} column. This list is passed to the transformer model.
    """
    original_df['added_id'] = original_df.index + 1
    filtered_df = original_df.loc[pd.notnull(original_df[text_column])]
    print(f"Filtered dataframe for {text_column}.")
    print(f"Dataframe has size {filtered_df.shape}.")
    
    item_list = filtered_df[text_column].to_list()
    print(f"The list has length {len(item_list)}.")
        
    return filtered_df, item_list

In [14]:
def remove_name(text):
    # nlp = spacy.load("en_core_web_lg")

    doc = nlp(text)
    
    names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    #print(names)  

    if names:
        new_txt = text
        for name in names:
            new_txt = new_txt.replace(name, "")
            #print(new_txt)
        
        return new_txt
    
    else:
        return text

In [15]:
def sentence_segmenter(data_frame, text_column):
    # first, create list from series of text
    
    entry_list = data_frame[text_column].to_list()
    
    # try using dictionary of lists
    
    new_df_dict = {'text_column': [],
                   'job_id': [],
                  #  'original_entry': [],
                   'split_sent': [],
                   'sent_num': []}
    
    # nlp = English()
    # nlp = spacy.load("en_core_web_lg")
    nlp.add_pipe('sentencizer')
    
    ## using the process of converting series to list and then iterating over list
    for i, entry in enumerate(entry_list):
      doc = nlp(entry)
      sentences = [sent.text.strip() for sent in doc.sents]
      if i % 1000 == 0:
        print(f"finished {i} entries.")

      try:  
        for j, sent in enumerate(sentences):
          # print(sent)
          # new_sent = remove_name(sent)
          new_df_dict['text_column'].append(text_column)
          new_df_dict['job_id'].append(data_frame['job_id'].iloc[i])
          # new_df_dict['original_entry'].append(entry)
          new_df_dict['split_sent'].append(sent)
          new_df_dict['sent_num'].append(j)

      except:
        print(f"Entry {i} sentence {j} failed")
          
    sentence_df = pd.DataFrame(new_df_dict)
    
    item_list = sentence_df['split_sent'].to_list()
    
    print(f"text column length {len(new_df_dict['text_column'])}")
    print(f"job id length {len(new_df_dict['job_id'])}")
    print(f"split sent length {len(new_df_dict['split_sent'])}")
    print(f"sent num length {len(new_df_dict['sent_num'])}")


    print(f"Filtered dataframe for {text_column}.")
    print(f"Dataframe has size {sentence_df.shape}.")
    print(f"The list has length {len(item_list)}.")

    
    return sentence_df, item_list
    


### Data Import

In [16]:
os.listdir()
proj_path = "/content/drive/Shareddrives/Project - Engineering Jobs and Non-academic Career Pathways"
os.chdir(proj_path)
os.listdir()

['exp Jobs analysis - similar tasks and activities.ipynb',
 'exp Jobs analysis - work activities clusterings.ipynb',
 '[OLD] Jobs analysis.ipynb',
 'sustainability_US_20210411.csv',
 'eng_jobs.Rmd',
 'jobs_text.py',
 'bme_top50_skills.png',
 'project engineering jobs.Rproj',
 'sus_jobs.Rmd',
 'sustainability jobs map.png',
 '.Rhistory',
 '.gitignore',
 'paper - engineering jobs.docx',
 'eng_jobs.py',
 '.RData',
 'sustainability_Australia.csv',
 'civil_top50_skills.png',
 'environmental_engineering_Australia.csv',
 'Sustainability in Engineering Job Skills.png',
 'civ_ed_grad_df_20210916.csv',
 'biomedical_us_20210410.csv',
 'sustainability_cortext_20210411.csv',
 'sustainability_us_20210409.csv',
 'sustainability_UK.csv',
 'chem_eng_20210409.csv',
 'mechanical_us_20210411.csv',
 'civ_eng_20210409.csv',
 'environmental_us_20210410.csv',
 'mechanical_engineering_US.csv',
 'sustainability_cortext_20210411.zip',
 'electrical_engineering_US.csv',
 'sustainability jobs',
 '.Rproj.user',
 '.g

In [17]:
# jobs_df = pd.read_csv("biomedical_us_20210410.csv")
jobs_df = pd.read_csv("mechanical_us_20210411.csv")

In [18]:
jobs_df = jobs_df.rename(columns={'Unnamed: 0': 'job_id'})
jobs_df

,job_id,title,company,location,salary,education,skills,description
0,0,"Chemical, Mechanical or Industrial Engineering...",BECHT,Remote,"About $40,000 - $55,000 a year","[""Bachelor's degree"", ""Master's degree""]","['TWIC Card', 'Microsoft Excel', 'CAD', 'Bluep...","Becht is a 5 STAR rated Glassdoor, a Top 100 B..."
1,1,2021 SUMMER/FALL MECHANICAL ENGINEERING CO-OP,Iowa Fertilizer Company,"Wever, IA","About $50,000 - $69,000 a year","[""Bachelor's degree""]","['Analysis skills', ""Driver's License"", 'Micro...",Iowa Fertilizer Company (IFCo) is looking for ...
2,2,Maintenance Engineer PM,East West,"Vail, CO",$20 an hour,[],['Solaris'],Description\nAre you interested in working for...
3,3,Refrigeration Engineer,Undisclosed Manufacture,"Mocksville, NC","$55,000 - $75,000 a year","[""Bachelor's degree""]","['SolidWorks', 'Mechanical engineering', 'Data...",R&D analysis and data collection\nStudy energy...
4,4,Associate Engineer,Moog Inc.,"Blacksburg, VA","About $69,000 - $92,000 a year","[""Bachelor's degree"", ""Associate's degree""]","['Communication skills', 'Microsoft Office', '...",Moog Inc is looking for an entry level Associa...
...,...,...,...,...,...,...,...,...
42925,42925,BIM Engineer,"Torcon, Inc","Red Bank, NJ","$75,000 - $130,000 a year","[""Bachelor's degree""]","['Revit', 'Sketch', 'Project engineering', 'Sk...",BIM ENGINEER\nTorcon is seeking a BIM Engineer...
42926,42926,Medical Device (Mechanical) Engineer,Delpor Inc,"Brisbane, CA","$65,000 - $85,000 a year","[""Bachelor's degree"", ""Master's degree""]","['Time management', 'Clinical Nurse Specialist...",Medical Device (Mechanical) Engineer\nNo Recru...
42927,42927,Quality Engineer,Inspection Engineering,"Sharonville, OH","About $56,000 - $78,000 a year","[""Associate's degree""]","['Coordinate measuring machine', 'SolidWorks',...","Will be responsible for sales demonstrations, ..."
42928,42928,Mechanical Engineer HVAC Consulting,Fiskaa Engineering,"New York, NY","About $60,000 - $77,000 a year","[""Bachelor's degree""]","['Revit', 'Microsoft Excel', 'CAD', 'Communica...","Fiskaa Engineering, a professional consulting ..."


In [19]:
batch_size = 200
jobs_df['batch_num'] = jobs_df.apply(lambda row: int(row['job_id']/batch_size), axis=1)

In [20]:
jobs_df

,job_id,title,company,location,salary,education,skills,description,batch_num
0,0,"Chemical, Mechanical or Industrial Engineering...",BECHT,Remote,"About $40,000 - $55,000 a year","[""Bachelor's degree"", ""Master's degree""]","['TWIC Card', 'Microsoft Excel', 'CAD', 'Bluep...","Becht is a 5 STAR rated Glassdoor, a Top 100 B...",0
1,1,2021 SUMMER/FALL MECHANICAL ENGINEERING CO-OP,Iowa Fertilizer Company,"Wever, IA","About $50,000 - $69,000 a year","[""Bachelor's degree""]","['Analysis skills', ""Driver's License"", 'Micro...",Iowa Fertilizer Company (IFCo) is looking for ...,0
2,2,Maintenance Engineer PM,East West,"Vail, CO",$20 an hour,[],['Solaris'],Description\nAre you interested in working for...,0
3,3,Refrigeration Engineer,Undisclosed Manufacture,"Mocksville, NC","$55,000 - $75,000 a year","[""Bachelor's degree""]","['SolidWorks', 'Mechanical engineering', 'Data...",R&D analysis and data collection\nStudy energy...,0
4,4,Associate Engineer,Moog Inc.,"Blacksburg, VA","About $69,000 - $92,000 a year","[""Bachelor's degree"", ""Associate's degree""]","['Communication skills', 'Microsoft Office', '...",Moog Inc is looking for an entry level Associa...,0
...,...,...,...,...,...,...,...,...,...
42925,42925,BIM Engineer,"Torcon, Inc","Red Bank, NJ","$75,000 - $130,000 a year","[""Bachelor's degree""]","['Revit', 'Sketch', 'Project engineering', 'Sk...",BIM ENGINEER\nTorcon is seeking a BIM Engineer...,214
42926,42926,Medical Device (Mechanical) Engineer,Delpor Inc,"Brisbane, CA","$65,000 - $85,000 a year","[""Bachelor's degree"", ""Master's degree""]","['Time management', 'Clinical Nurse Specialist...",Medical Device (Mechanical) Engineer\nNo Recru...,214
42927,42927,Quality Engineer,Inspection Engineering,"Sharonville, OH","About $56,000 - $78,000 a year","[""Associate's degree""]","['Coordinate measuring machine', 'SolidWorks',...","Will be responsible for sales demonstrations, ...",214
42928,42928,Mechanical Engineer HVAC Consulting,Fiskaa Engineering,"New York, NY","About $60,000 - $77,000 a year","[""Bachelor's degree""]","['Revit', 'Microsoft Excel', 'CAD', 'Communica...","Fiskaa Engineering, a professional consulting ...",214


##### Read in Onet Task Data

In [21]:
# task statements from https://www.onetcenter.org/dictionary/26.2/excel/task_statements.html
# onet_tasks = pd.read_csv("Task Statements.csv")
onet_tasks = pd.read_csv("onet_tasks_clustered.csv")
onet_activities = pd.read_csv("onet_activities_clustered.csv")

In [22]:

onet_tasks

,sent_id,O*NET-SOC Code,Title,Task ID,Task,Task Type,Incumbents Responding,Date,Domain Source,cluster_label
0,0,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,Core,87.0,07/2014,Incumbent,485
1,1,11-1011.00,Chief Executives,8831,Appoint department heads or managers and assig...,Core,87.0,07/2014,Incumbent,211
2,2,11-1011.00,Chief Executives,8825,Analyze operations to evaluate performance of ...,Core,87.0,07/2014,Incumbent,115
3,3,11-1011.00,Chief Executives,8826,"Direct, plan, or implement policies, objective...",Core,87.0,07/2014,Incumbent,485
4,4,11-1011.00,Chief Executives,8827,"Prepare budgets for approval, including those ...",Core,87.0,07/2014,Incumbent,173
...,...,...,...,...,...,...,...,...,...,...
17971,19254,53-7121.00,"Tank Car, Truck, and Ship Loaders",12807,Unload cars containing liquids by connecting h...,Supplemental,85.0,08/2019,Incumbent,253
17972,19255,53-7121.00,"Tank Car, Truck, and Ship Loaders",12804,"Clean interiors of tank cars or tank trucks, u...",Supplemental,85.0,08/2019,Incumbent,313
17973,19256,53-7121.00,"Tank Car, Truck, and Ship Loaders",12803,Lower gauge rods into tanks or read meters to ...,Supplemental,85.0,08/2019,Incumbent,197
17974,19257,53-7121.00,"Tank Car, Truck, and Ship Loaders",12805,Operate conveyors and equipment to transfer gr...,Supplemental,85.0,08/2019,Incumbent,290


In [23]:
onet_activities

,sent_id,Element ID,Element Name,IWA ID,IWA Title,DWA ID,DWA Title,cluster_label
0,0,4.A.1.a.1,Getting Information,4.A.1.a.1.I01,Study details of artistic productions.,4.A.1.a.1.I01.D01,Review art or design materials.,14
1,1,4.A.1.a.1,Getting Information,4.A.1.a.1.I01,Study details of artistic productions.,4.A.1.a.1.I01.D02,Study details of musical compositions.,28
2,2,4.A.1.a.1,Getting Information,4.A.1.a.1.I01,Study details of artistic productions.,4.A.1.a.1.I01.D03,Review production information to determine cos...,40
3,3,4.A.1.a.1,Getting Information,4.A.1.a.1.I01,Study details of artistic productions.,4.A.1.a.1.I01.D04,Study scripts to determine project requirements.,48
4,4,4.A.1.a.1,Getting Information,4.A.1.a.1.I01,Study details of artistic productions.,4.A.1.a.1.I01.D05,Review audio or video recordings.,9
...,...,...,...,...,...,...,...,...
2082,2082,4.A.4.c.3,Monitoring and Controlling Resources,4.A.4.c.3.I07,Monitor resources or inventories.,4.A.4.c.3.I07.D01,Monitor availability of equipment or supplies.,47
2083,2083,4.A.4.c.3,Monitoring and Controlling Resources,4.A.4.c.3.I07,Monitor resources or inventories.,4.A.4.c.3.I07.D02,Inventory materials or equipment.,25
2084,2084,4.A.4.c.3,Monitoring and Controlling Resources,4.A.4.c.3.I07,Monitor resources or inventories.,4.A.4.c.3.I07.D03,Inventory medical supplies or equipment.,25
2085,2085,4.A.4.c.3,Monitoring and Controlling Resources,4.A.4.c.3.I07,Monitor resources or inventories.,4.A.4.c.3.I07.D04,Monitor inventories of products or materials.,84


Create unique tasks dataframe and list of tasks

In [24]:
unique_tasks_df = onet_tasks.drop_duplicates(subset=['Task'])

In [25]:
# unique_tasks_df.reset_index(inplace=True)
unique_tasks_df = unique_tasks_df.rename(columns = {'Task': 'onet_text'})

In [26]:
unique_tasks_df['onet_type'] = 'task'

In [27]:
# unique_tasks = unique_tasks_df['Task'].to_list()

In [28]:
# unique_tasks

Create unique activities dataframe and list of activities

In [29]:
unique_activities_df = onet_activities.drop_duplicates(subset=['DWA Title'])

In [30]:
# unique_activities_df.reset_index(inplace=True)
unique_activities_df = unique_activities_df.rename(columns = {'DWA Title': 'onet_text'})

In [31]:
unique_activities_df['onet_type'] = 'activity'

In [32]:
# unique_activities_list = unique_activities_df['DWA Title'].to_list()

In [33]:
# print(len(unique_activities_list))

Join the activities and tasks into one dataframe

First need to subset the dataframes and make sure they have the same column headers

In [34]:
tasks_join_df = unique_tasks_df[['sent_id', 'onet_type', 'onet_text', 'cluster_label']]
activities_join_df = unique_activities_df[['sent_id', 'onet_type', 'onet_text', 'cluster_label']]

In [35]:
onet_joined_df = pd.concat([tasks_join_df, activities_join_df])

In [36]:
onet_joined_df

,sent_id,onet_type,onet_text,cluster_label
0,0,task,Direct or coordinate an organization's financi...,485
1,1,task,Appoint department heads or managers and assig...,211
2,2,task,Analyze operations to evaluate performance of ...,115
3,3,task,"Direct, plan, or implement policies, objective...",485
4,4,task,"Prepare budgets for approval, including those ...",173
...,...,...,...,...
2082,2082,activity,Monitor availability of equipment or supplies.,47
2083,2083,activity,Inventory materials or equipment.,25
2084,2084,activity,Inventory medical supplies or equipment.,25
2085,2085,activity,Monitor inventories of products or materials.,84


In [37]:
onet_text_list = onet_joined_df.onet_text.to_list()

###### Task embeddings

In [38]:
embed_tasks=False

In [39]:
if embed_tasks:
  task_embeddings = ec.embed_raw_text(onet_text_list, 'all-miniLM', max_seq_length=200)

In [40]:
# pickle the embeddings
if embed_tasks:
  pickle_out = open(f'onet_text_miniLM.pickle', 'wb')
  pickle.dump(task_embeddings, pickle_out)
  pickle_out.close()

In [41]:
pickle_in = open(f'onet_text_miniLM.pickle', 'rb')
task_embeddings = pickle.load(pickle_in)
pickle_in.close()

###### Clean the descriptions to replace new line character with period

In [42]:
# test_df['clean_description'] = test_df['description'].str.replace("\n", ". ")

In [43]:
#check to make sure new line character replaced
# test_df['clean_description'].to_list()[0]

###### Split Description by Sentence

Break description into sentences with the sentence_segumenter function

In [44]:
# nlp = spacy.load("en_core_web_lg")

In [45]:
# description_sent_df, description_sent_list = sentence_segmenter(test_df, 'clean_description')

In [46]:
# test_df.shape

In [47]:
# description_sent_df.columns

In [48]:
# description_sent_df.reset_index(inplace=True)
# description_sent_df = description_sent_df.rename(columns = {'index':'sent_id'})

In [49]:
# calculate similarity
from scipy.spatial.distance import cdist

In [50]:
n = 3
threshold = 0.7

In [51]:
# n = 7




sim_onet_dict = {'batch_num': [],
                  'job_id':[],
                  'sent_id':[],
                  'original_sent_text':[],
                  'similar_onet_id':[],
                  'similarity_rank':[],
                  'similarity_score':[],
                  'similar_onet_text':[],
                  'similar_onet_cluster_label': [],
                 'similar_onet_type':[]}


# for i in np.sort(unlabeled_sent_df['cluster_label'].unique()):
#for i in range(10):

for i in range(4):
  test_df = jobs_df[jobs_df['batch_num'] == i]

  test_df['clean_description'] = test_df['description'].str.replace("\n", ". ")
  
  nlp = spacy.load("en_core_web_lg")
  if i > 0:
    del description_sent_df
    del description_sent_list
  description_sent_df, description_sent_list = sentence_segmenter(test_df, 'clean_description')

  description_sent_df.reset_index(inplace=True)
  description_sent_df = description_sent_df.rename(columns = {'index':'sent_id'})

  print(f"working on batch {i}.")

  # try:
  # print(f"\nWorking on cluster {i}")
  # temp_cl_df = unlabeled_sent_df[unlabeled_sent_df['cluster_label'] == i]

  description_sent_list = description_sent_df.split_sent.to_list()

  if i > 0:
    del description_sent_embeddings
  description_sent_embeddings = ec.embed_raw_text(description_sent_list, 'all-miniLM', max_seq_length=200)

  print(description_sent_embeddings.shape)

  print("Calculating similarity scores")
  cosine_df = 1 - cdist(description_sent_embeddings, task_embeddings, metric='cosine')

  if i > 0:
    del scores_df
  scores_df = pd.DataFrame(cosine_df)

  for j in range(scores_df.shape[0]):
    # top_n_scores = scores_df.iloc[j,:].nlargest(n)
    # indices = top_n_scores.index
    
    #print(top_n_scores)
    # pick scores over threshold
    scores = scores_df.iloc[j,:]
    indices = scores[scores > threshold].index


    
    #print(indices)


    for num, k in enumerate(indices):
      # print(num)
      # print(i)
      # print(f"Item {j} had score {top_n_scores.iloc[num]}. The task was: {tasks_text[j]}")
      # print(f"The original classification of this task was {tasks_df['label'].iloc[j]}")

      #print(f"Cluster label: {temp_cl_df['cluster_label'].iloc[j]}")
      #print(labeled_sent_df['sum_sent'].iloc[k])
      #print(temp_cl_df['split_sent'].iloc[j])

      sim_onet_dict['batch_num'].append(i)
      sim_onet_dict['job_id'].append(description_sent_df['job_id'].iloc[j])
      sim_onet_dict['sent_id'].append(description_sent_df['sent_id'].iloc[j])
      sim_onet_dict['original_sent_text'].append(description_sent_df['split_sent'].iloc[j])
      # sim_sents_dict['original_cluster_label'].append(temp_cl_df['cluster_label'].iloc[j])
      sim_onet_dict['similarity_rank'].append(num)
      sim_onet_dict['similar_onet_text'].append(onet_joined_df['onet_text'].iloc[k])
      sim_onet_dict['similar_onet_type'].append(onet_joined_df['onet_type'].iloc[k])
      sim_onet_dict['similar_onet_id'].append(onet_joined_df['sent_id'].iloc[k])
      sim_onet_dict['similarity_score'].append(scores.iloc[k])
      sim_onet_dict['similar_onet_cluster_label'].append(onet_joined_df['cluster_label'].iloc[k])
      # sim_sents_dict['similar_sent_label'].append(unique_tasks_df['manual_text_label_v2'].iloc[k]) # may need to make this manual_text_label instead of codebook_v2



  # except:
  #   print(f"Error encountered in batch {i}.")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


finished 0 entries.
text column length 9073
job id length 9073
split sent length 9073
sent num length 9073
Filtered dataframe for clean_description.
Dataframe has size (9073, 4).
The list has length 9073.
working on batch 0.
Original model max_seq_length: 128.
New model max_seq_length: 200.
Raw text embedding completed.
Original raw text had dimension: 9073.
The dimension of the embedding is (9073, 384).
(9073, 384)
Calculating similarity scores
finished 0 entries.
text column length 9284
job id length 9284
split sent length 9284
sent num length 9284
Filtered dataframe for clean_description.
Dataframe has size (9284, 4).
The list has length 9284.
working on batch 1.
Original model max_seq_length: 128.
New model max_seq_length: 200.
Raw text embedding completed.
Original raw text had dimension: 9284.
The dimension of the embedding is (9284, 384).
(9284, 384)
Calculating similarity scores
finished 0 entries.
text column length 9163
job id length 9163
split sent length 9163
sent num lengt

In [52]:
sim_onet_df = pd.DataFrame(sim_onet_dict)
sim_onet_df

,batch_num,job_id,sent_id,original_sent_text,similar_onet_id,similarity_rank,similarity_score,similar_onet_text,similar_onet_cluster_label,similar_onet_type
0,0,0,12,Supporting engineers by completing engineering...,4078,0,0.713755,"Assist engineers in developing, building, or t...",208,task
1,0,0,15,"Create Excel, Word, and PowerPoint summaries a...",13337,0,0.705085,"Prepare invoices, reports, memos, letters, fin...",593,task
2,0,0,31,Access to further training and development.,4897,0,0.719498,Identify training and development needs.,30,task
3,0,0,31,Access to further training and development.,5509,1,0.704984,Attend training sessions to increase knowledge...,373,task
4,0,0,31,Access to further training and development.,1465,2,0.728870,Develop training materials.,26,activity
...,...,...,...,...,...,...,...,...,...,...
17495,3,797,9541,Review and perform quality assurance for desig...,478,0,0.714581,Evaluate designs or specifications to ensure q...,86,activity
17496,3,797,9549,Provide technical engineering expertise to res...,3471,0,0.700578,Provide technical expertise or support related...,421,task
17497,3,797,9565,Mentors other engineers and technical professi...,3599,0,0.718628,Provide technical direction to other engineers...,421,task
17498,3,797,9573,Required to work safely and follow all Entergy...,15615,0,0.745212,"Comply with all applicable standards, policies...",413,task


In [53]:
# scores = scores_df.iloc[5,:]
# scores[scores >0.7].index


In [54]:
threshold = 0.7
most_sim_df = sim_onet_df[sim_onet_df['similarity_score'] > threshold]

In [55]:
most_sim_df

,batch_num,job_id,sent_id,original_sent_text,similar_onet_id,similarity_rank,similarity_score,similar_onet_text,similar_onet_cluster_label,similar_onet_type
0,0,0,12,Supporting engineers by completing engineering...,4078,0,0.713755,"Assist engineers in developing, building, or t...",208,task
1,0,0,15,"Create Excel, Word, and PowerPoint summaries a...",13337,0,0.705085,"Prepare invoices, reports, memos, letters, fin...",593,task
2,0,0,31,Access to further training and development.,4897,0,0.719498,Identify training and development needs.,30,task
3,0,0,31,Access to further training and development.,5509,1,0.704984,Attend training sessions to increase knowledge...,373,task
4,0,0,31,Access to further training and development.,1465,2,0.728870,Develop training materials.,26,activity
...,...,...,...,...,...,...,...,...,...,...
17495,3,797,9541,Review and perform quality assurance for desig...,478,0,0.714581,Evaluate designs or specifications to ensure q...,86,activity
17496,3,797,9549,Provide technical engineering expertise to res...,3471,0,0.700578,Provide technical expertise or support related...,421,task
17497,3,797,9565,Mentors other engineers and technical professi...,3599,0,0.718628,Provide technical direction to other engineers...,421,task
17498,3,797,9573,Required to work safely and follow all Entergy...,15615,0,0.745212,"Comply with all applicable standards, policies...",413,task


In [56]:
# most_sim_df = pd.merge(most_sim_df, onet_joined_df, how="left", left_on="similar_task_text", right_on="Task")

In [57]:
# most_sim_df

In [58]:
most_sim_df.to_csv("most_sim_onet_df_mechanical_batched.csv", index=False)